In [24]:
train_files = ['MSRpar', 'MSRvid', 'SMTeuroparl']
data = []
labels = []
for file in train_files:
    with open('train/STS.input.'+file+'.txt', 'r') as f:
        data += [sent.split('\t') for sent in f.readlines()]
    with open('train/STS.gs.'+file+'.txt', 'r') as f:
        labels += [float(num) for num in f.readlines()]